In [2]:
import numpy as np
import pandas as pd

In [3]:
item_category_df = pd.read_csv("../data/item_categories_trans.csv")
items_df = pd.read_csv("../data/items.csv")
train_df = pd.read_csv("../data/sales_train.csv")
test_df = pd.read_csv("../data/test.csv")

In [4]:
shop_df = pd.read_csv('../data/shops_trans.csv', encoding='utf-8')
shop_df = shop_df.drop(["shop_name"], axis=1)
shop_df.head()

,shop_id,en_shop
0,0,"Yakutsk Ordzhonikidze, 56 francs"
1,1,"Yakutsk TC ""Central"" franc"
2,2,"Adygea Shopping Center ""Mega"""
3,3,"Balashikha TRK ""October-Kinomir"""
4,4,"Volzhsky shopping center ""Volga Mall"""


In [5]:
def split(name):
    sptname = name.split(' "')
    lastspt = sptname[0].split(" ")
    return lastspt[0]
shop_df["Region"] = list(map(split, shop_df.en_shop))
shop_df.iloc[8:13]

,shop_id,en_shop,Region
8,8,"Voronezh TRC City-Park ""Grad""",Voronezh
9,9,Outbound Trade,Outbound
10,10,Zhukovsky Str. Chkalov 39m?,Zhukovsky
11,11,Zhukovsky Str. Chkalov 39 m²,Zhukovsky
12,12,Online shop Emergency,Online


In [6]:
working_df = train_df.merge(shop_df, on="shop_id")
working_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,en_shop,Region
0,02.01.2013,0,59,22154,999.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl
1,10.01.2013,0,59,22151,399.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl
2,04.01.2013,0,59,5603,699.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl
3,19.01.2013,0,59,5587,199.0,2.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl
4,31.01.2013,0,59,5613,5571.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl


In [7]:
findlast = working_df.groupby(["Region", "en_shop"])["date_block_num", "shop_id"].last()
findlast = findlast.reset_index().sort_values(by=["date_block_num"])
missing = findlast[findlast.date_block_num < 33]
shops = missing["shop_id"].tolist() 
shops

[1, 0, 23, 8, 32, 13, 43, 17, 40, 11, 30, 33, 54, 29, 51, 27]

In [8]:
date_df = pd.DataFrame(working_df["date"].str.split(".").tolist(),  columns = ['day','month','year'])
date_df.head()

,day,month,year
0,02,01,2013
1,10,01,2013
2,04,01,2013
3,19,01,2013
4,31,01,2013


In [9]:
working_df = working_df.merge(date_df[["year", "month"]], left_index=True, right_index=True)
del working_df["date_block_num"]
del working_df["date"]

In [10]:
working_df.head()

,shop_id,item_id,item_price,item_cnt_day,en_shop,Region,year,month
0,59,22154,999.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,2013,01
1,59,22151,399.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,2013,01
2,59,5603,699.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,2013,01
3,59,5587,199.0,2.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,2013,01
4,59,5613,5571.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,2013,01


In [11]:
## 각 카테고리별 큰카테고리 구하고 카테고리 df만들기
df_category = pd.read_csv("../data/item_categories_trans.csv")
df_category
del df_category["item_category_name"]
df_category["en_category"].replace(to_replace='Книги - Путеводители', value='Books - Guide', inplace=True)
df_category["en_category"].replace(to_replace='Payment Cards - Windows (Digital)', value='Payment cards - Windows (Digital)', inplace=True)
df_category["en_category"].replace(to_replace='Payment cards (Movies, Music, Games)', value='Payment cards - (Movies, Music, Games)', inplace=True)
df_category.head()

,item_category_id,en_category
0,0,PC - Headsets / Headphones
1,1,Accessories - PS2
2,2,Accessories - PS3
3,3,Accessories - PS4
4,4,Accessories - PSP


In [12]:
def splitcategory(x):
    if '-' in x:
        a = x.split('-')
        for i in a:
            i.strip()
        return a[0]
    else:
        return x

In [13]:
df_category["big_cate"]=list(map(splitcategory, df_category["en_category"]))
df_big_category = df_category.groupby(["big_cate", "en_category"])["item_category_id"].first().reset_index()
print(df_big_category.shape)
df_big_category.head()

(84, 3)


,big_cate,en_category,item_category_id
0,Accessories,Accessories - PS2,1
1,Accessories,Accessories - PS3,2
2,Accessories,Accessories - PS4,3
3,Accessories,Accessories - PSP,4
4,Accessories,Accessories - PSVita,5


In [14]:
working_df.isnull().sum()

shop_id         0
item_id         0
item_price      0
item_cnt_day    0
en_shop         0
Region          0
year            0
month           0
dtype: int64

In [15]:
item_with_bigcate = pd.merge(items_df, df_big_category, on="item_category_id", how="inner")
item_with_bigcate.head()

,item_name,item_id,item_category_id,big_cate,en_category
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Cinema,Cinema - DVD
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Cinema,Cinema - DVD
2,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Cinema,Cinema - DVD
3,***КОРОБКА (СТЕКЛО) D,4,40,Cinema,Cinema - DVD
4,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,5,40,Cinema,Cinema - DVD


In [16]:
pre_data_with_cate = pd.merge(working_df, item_with_bigcate[["item_id", "item_category_id", "big_cate", "en_category"]], on="item_id", how="inner"  )
pre_data_with_cate.head()

,shop_id,item_id,item_price,item_cnt_day,en_shop,Region,year,month,item_category_id,big_cate,en_category
0,59,22154,999.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,2013,01,37,Cinema,Cinema - Blu-Ray
1,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",Moscow,2013,01,37,Cinema,Cinema - Blu-Ray
2,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",Moscow,2013,01,37,Cinema,Cinema - Blu-Ray
3,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",Moscow,2013,01,37,Cinema,Cinema - Blu-Ray
4,25,22154,999.0,1.0,"Moscow TRK ""Atrium""",Moscow,2013,01,37,Cinema,Cinema - Blu-Ray


## 각 달의 상점별 BIG_cate별 판매량에 각 달의 3개년 평균 item_en_cate 비중을 계산한후, 이걸 변수로 쓰자

In [17]:
# 일단  각 달의 상저별 Big cate별 판매량 
base = pre_data_with_cate.groupby(["shop_id", "year", "month","big_cate"])["item_cnt_day"].sum()
base = base.reset_index()
base.head()

,shop_id,year,month,big_cate,item_cnt_day
0,0,2013,01,Accessories,190.0
1,0,2013,01,Books,70.0
2,0,2013,01,Cinema,2276.0
3,0,2013,01,Clean carriers (spire),24.0
4,0,2013,01,Clean media (piece),86.0


In [18]:
#상점별 각 달의 평균 판매량 
avgmonth = pre_data_with_cate.groupby(["shop_id", "year", "month","en_category", "big_cate"])["item_cnt_day"].sum()
avgmonth = avgmonth.reset_index()
avgmonth.head()

,shop_id,year,month,en_category,big_cate,item_cnt_day
0,0,2013,01,Accessories - PS3,Accessories,53.0
1,0,2013,01,Accessories - PS4,Accessories,28.0
2,0,2013,01,Accessories - PSP,Accessories,16.0
3,0,2013,01,Accessories - PSVita,Accessories,28.0
4,0,2013,01,Accessories - XBOX 360,Accessories,65.0


In [19]:
yearmonth = lambda x, y, z : str(x) + str(y) + str(z)
base["yearmonth"] = list(map(yearmonth, base["year"], base["month"], base["big_cate"]))
avgmonth["yearmonth"] = list(map(yearmonth, avgmonth["year"], avgmonth["month"], avgmonth["big_cate"]))

In [20]:
avgmonth.rename(columns={"item_cnt_day" : "item_cnt_smllcate"}, inplace=True)
avgmonth.head()


,shop_id,year,month,en_category,big_cate,item_cnt_smllcate,yearmonth
0,0,2013,01,Accessories - PS3,Accessories,53.0,201301Accessories
1,0,2013,01,Accessories - PS4,Accessories,28.0,201301Accessories
2,0,2013,01,Accessories - PSP,Accessories,16.0,201301Accessories
3,0,2013,01,Accessories - PSVita,Accessories,28.0,201301Accessories
4,0,2013,01,Accessories - XBOX 360,Accessories,65.0,201301Accessories


In [21]:
base.rename(columns={"item_cnt_day" : "item_cnt_bigcate"}, inplace=True)
base.head()

,shop_id,year,month,big_cate,item_cnt_bigcate,yearmonth
0,0,2013,01,Accessories,190.0,201301Accessories
1,0,2013,01,Books,70.0,201301Books
2,0,2013,01,Cinema,2276.0,201301Cinema
3,0,2013,01,Clean carriers (spire),24.0,201301Clean carriers (spire)
4,0,2013,01,Clean media (piece),86.0,201301Clean media (piece)


In [22]:
avgbase_df = avgmonth.merge(base[["yearmonth", "item_cnt_bigcate"]], on="yearmonth", how="right")
avgbase_df = avgbase_df.fillna(0)
avgbase_df.head()

,shop_id,year,month,en_category,big_cate,item_cnt_smllcate,yearmonth,item_cnt_bigcate
0,0,2013,01,Accessories - PS3,Accessories,53.0,201301Accessories,190.0
1,0,2013,01,Accessories - PS4,Accessories,28.0,201301Accessories,190.0
2,0,2013,01,Accessories - PSP,Accessories,16.0,201301Accessories,190.0
3,0,2013,01,Accessories - PSVita,Accessories,28.0,201301Accessories,190.0
4,0,2013,01,Accessories - XBOX 360,Accessories,65.0,201301Accessories,190.0


In [23]:
allinone_df = avgbase_df.groupby(["month", "en_category", "big_cate"])["item_cnt_smllcate", "item_cnt_bigcate"].sum()
allinone_df = allinone_df.reset_index()
allinone_df.head()

,month,en_category,big_cate,item_cnt_smllcate,item_cnt_bigcate
0,01,Accessories - PS2,Accessories,44.0,3575.0
1,01,Accessories - PS3,Accessories,137836.0,483765.0
2,01,Accessories - PS4,Accessories,138384.0,479910.0
3,01,Accessories - PSP,Accessories,15748.0,282135.0
4,01,Accessories - PSVita,Accessories,48232.0,422305.0


In [24]:
#각 달에서 small cate와 big cate의 비중을 이제 계산할수 있어!
#각 달별 평균 판매량의 가중치 
devide = lambda x, y: x /y

allinone_df["weight"] = list(map(devide, allinone_df.item_cnt_smllcate, allinone_df.item_cnt_bigcate))
allinone_df.head()

,month,en_category,big_cate,item_cnt_smllcate,item_cnt_bigcate,weight
0,01,Accessories - PS2,Accessories,44.0,3575.0,0.012308
1,01,Accessories - PS3,Accessories,137836.0,483765.0,0.284923
2,01,Accessories - PS4,Accessories,138384.0,479910.0,0.288354
3,01,Accessories - PSP,Accessories,15748.0,282135.0,0.055817
4,01,Accessories - PSVita,Accessories,48232.0,422305.0,0.114211


In [25]:
# 전체에서 각 월별로 각 카테고리가 얼마나 차지하는지 만들기
#일단 각 월별 전체 판매량
cateportion_month_df = pre_data_with_cate.groupby(["month"])["item_cnt_day"].sum()
cateportion_month_df = cateportion_month_df.reset_index()
cateportion_month_df.rename(columns={"item_cnt_day": "total"}, inplace=True)
cateportion_month_df.head()

,month,total
0,01,359349.0
1,02,321975.0
2,03,344453.0
3,04,281573.0
4,05,277055.0


In [26]:
cateportion_df = pre_data_with_cate.groupby(["month", "big_cate"])["item_cnt_day"].sum()
cateportion_df = cateportion_df.reset_index()
cateportion_df.head()

,month,big_cate,item_cnt_day
0,01,Accessories,10770.0
1,01,Android games,1.0
2,01,Books,6167.0
3,01,Cinema,98632.0
4,01,Clean carriers (spire),187.0


In [27]:
cateportion_df = cateportion_df.merge(cateportion_month_df, on="month", how="left")
cateportion_df.head()

,month,big_cate,item_cnt_day,total
0,01,Accessories,10770.0,359349.0
1,01,Android games,1.0,359349.0
2,01,Books,6167.0,359349.0
3,01,Cinema,98632.0,359349.0
4,01,Clean carriers (spire),187.0,359349.0


In [28]:
cateportion_df["weightontotal"] = list(map(lambda x,y : x /y , cateportion_df.item_cnt_day, cateportion_df.total))
cateportion_df["bigcate_id"] = list(map(lambda x,y : str(x) + str(y), cateportion_df.month, cateportion_df.big_cate))
cateportion_df.head()
#저기있는 weightontotlal을 변수로 사용한다


,month,big_cate,item_cnt_day,total,weightontotal,bigcate_id
0,01,Accessories,10770.0,359349.0,0.029971,01Accessories
1,01,Android games,1.0,359349.0,0.000003,01Android games
2,01,Books,6167.0,359349.0,0.017162,01Books
3,01,Cinema,98632.0,359349.0,0.274474,01Cinema
4,01,Clean carriers (spire),187.0,359349.0,0.000520,01Clean carriers (spire)


In [29]:
#카테고리 ID 붙여주기
allinone_df = allinone_df.merge(df_category[["item_category_id", "en_category"]], on="en_category", how="inner")

In [30]:
allinone_df.head()

,month,en_category,big_cate,item_cnt_smllcate,item_cnt_bigcate,weight,item_category_id
0,01,Accessories - PS2,Accessories,44.0,3575.0,0.012308,1
1,02,Accessories - PS2,Accessories,45.0,2739.0,0.016429,1
2,01,Accessories - PS3,Accessories,137836.0,483765.0,0.284923,2
3,02,Accessories - PS3,Accessories,100320.0,365783.0,0.274261,2
4,03,Accessories - PS3,Accessories,86100.0,305595.0,0.281745,2


In [31]:
allinone_df["smallcateid"]= list(map(lambda x,y : str(x) +"-"+ str(y), allinone_df.month, allinone_df.item_category_id))
allinone_df["bigcate_id"] = list(map(lambda x,y : str(x) + str(y), allinone_df.month, allinone_df.big_cate))
allinone_df.head()

,month,en_category,big_cate,item_cnt_smllcate,item_cnt_bigcate,weight,item_category_id,smallcateid,bigcate_id
0,01,Accessories - PS2,Accessories,44.0,3575.0,0.012308,1,01-1,01Accessories
1,02,Accessories - PS2,Accessories,45.0,2739.0,0.016429,1,02-1,02Accessories
2,01,Accessories - PS3,Accessories,137836.0,483765.0,0.284923,2,01-2,01Accessories
3,02,Accessories - PS3,Accessories,100320.0,365783.0,0.274261,2,02-2,02Accessories
4,03,Accessories - PS3,Accessories,86100.0,305595.0,0.281745,2,03-2,03Accessories


In [32]:
allinone_df = allinone_df.merge(cateportion_df[["weightontotal", "bigcate_id"]], on="bigcate_id", how="left")
allinone_df.head()

,month,en_category,big_cate,item_cnt_smllcate,item_cnt_bigcate,weight,item_category_id,smallcateid,bigcate_id,weightontotal
0,01,Accessories - PS2,Accessories,44.0,3575.0,0.012308,1,01-1,01Accessories,0.029971
1,02,Accessories - PS2,Accessories,45.0,2739.0,0.016429,1,02-1,02Accessories,0.025626
2,01,Accessories - PS3,Accessories,137836.0,483765.0,0.284923,2,01-2,01Accessories,0.029971
3,02,Accessories - PS3,Accessories,100320.0,365783.0,0.274261,2,02-2,02Accessories,0.025626
4,03,Accessories - PS3,Accessories,86100.0,305595.0,0.281745,2,03-2,03Accessories,0.020371


In [33]:
pre_data_with_cate_working = pre_data_with_cate.groupby(["shop_id", "month", "big_cate", "item_category_id"])["item_cnt_day"].median()
pre_data_with_cate_working = pre_data_with_cate_working.reset_index()
pre_data_with_cate_working.head()

,shop_id,month,big_cate,item_category_id,item_cnt_day
0,0,01,Accessories,2,1.0
1,0,01,Accessories,3,1.0
2,0,01,Accessories,4,1.0
3,0,01,Accessories,5,1.0
4,0,01,Accessories,6,1.0


In [34]:
pre_data_with_cate_working["smallcateid"]= list(map(lambda x,y : str(x) +"-"+ str(y), pre_data_with_cate_working.month, pre_data_with_cate_working.item_category_id))
pre_data_with_cate_working["bigcate_id"] = list(map(lambda x,y : str(x) + str(y), pre_data_with_cate_working.month, pre_data_with_cate_working.big_cate))
pre_data_with_cate_working.head()

,shop_id,month,big_cate,item_category_id,item_cnt_day,smallcateid,bigcate_id
0,0,01,Accessories,2,1.0,01-2,01Accessories
1,0,01,Accessories,3,1.0,01-3,01Accessories
2,0,01,Accessories,4,1.0,01-4,01Accessories
3,0,01,Accessories,5,1.0,01-5,01Accessories
4,0,01,Accessories,6,1.0,01-6,01Accessories


In [35]:
final_working_df = pre_data_with_cate_working.merge(allinone_df[["weightontotal", "bigcate_id"]], on="bigcate_id", how="right")
final_working_df.drop(["bigcate_id", "smallcateid", "big_cate"], axis=1, inplace=True)
final_working_df.head()

,shop_id,month,item_category_id,item_cnt_day,weightontotal
0,0,01,2,1.0,0.029971
1,0,01,3,1.0,0.029971
2,0,01,4,1.0,0.029971
3,0,01,5,1.0,0.029971
4,0,01,6,1.0,0.029971


In [36]:
final_working_df.shape

(217483, 5)

In [37]:
letget_mt = final_working_df.as_matrix()
letget_mt

array([[0, '01', 2, 1.0, 0.029970863979028744],
       [0, '01', 3, 1.0, 0.029970863979028744],
       [0, '01', 4, 1.0, 0.029970863979028744],
       ...,
       [55, '10', 27, 1.0, 3.267770955398194e-06],
       [55, '12', 26, 1.0, 1.4200632212146085e-05],
       [55, '12', 27, 1.0, 5.680252884858434e-06]], dtype=object)

In [38]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [39]:
columnum = [0, 1 ,2]
for column_index in  columnum:
    le = preprocessing.LabelEncoder()
    le.fit(letget_mt[:,column_index])
    letget_mt[:,column_index] = le.transform(letget_mt[:,column_index])
    del le
    
letget_mt

array([[0, 0, 2, 1.0, 0.029970863979028744],
       [0, 0, 3, 1.0, 0.029970863979028744],
       [0, 0, 4, 1.0, 0.029970863979028744],
       ...,
       [55, 9, 27, 1.0, 3.267770955398194e-06],
       [55, 11, 26, 1.0, 1.4200632212146085e-05],
       [55, 11, 27, 1.0, 5.680252884858434e-06]], dtype=object)

In [40]:
one_hot_attr = letget_mt[:,0:3]
one_hot_attr

array([[0, 0, 2],
       [0, 0, 3],
       [0, 0, 4],
       ...,
       [55, 9, 27],
       [55, 11, 26],
       [55, 11, 27]], dtype=object)

In [41]:
onehot_enc = OneHotEncoder()
onehot_enc.fit(one_hot_attr.astype(int))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [42]:
shop_month_categoty_values = onehot_enc.transform(one_hot_attr.astype(int))

In [43]:
shop_month_categoty_values

<217483x156 sparse matrix of type '<class 'numpy.float64'>'
	with 652449 stored elements in Compressed Sparse Row format>

In [44]:
from scipy.sparse import csr_matrix
reald_values = final_working_df["weightontotal"].values.reshape(-1,1)
print(reald_values[0:5])
csr_matrix(reald_values)

[[0.02997086]
 [0.02997086]
 [0.02997086]
 [0.02997086]
 [0.02997086]]


<217483x1 sparse matrix of type '<class 'numpy.float64'>'
	with 217483 stored elements in Compressed Sparse Row format>

In [45]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
minmax.fit(reald_values)
reald_values = minmax.transform(reald_values)

#판매량을 MINMAX에다가 넣어주고, 적용!

In [46]:
from scipy.sparse import hstack
total_csc_matrix = hstack([shop_month_categoty_values,csr_matrix(reald_values)], format='csr')
#가격에대한 정보를 가로로 MINMAX SCALE해서 붙여준다 (카테고리데이터는 ONEHOT, 가격은 MINMAX!)

In [47]:
y = final_working_df["item_cnt_day"].values 
#우리가 찾는 값은 Y, 따라서 이 값의 값들을 y값으로두고!

In [48]:
from sklearn.linear_model import Lasso,LinearRegression
lr_simple = LinearRegression().fit(total_csc_matrix, y)
lr_lasso = Lasso().fit(total_csc_matrix, y)

#가즈아아아아아아

In [49]:
test_df.drop(["ID"], axis=1, inplace=True)

In [50]:
test_df["month"] = 11


In [51]:
test_df.head()

,shop_id,item_id,month
0,5,5037,11
1,5,5320,11
2,5,5233,11
3,5,5232,11
4,5,5268,11


In [52]:
items_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [53]:
working = test_df.merge(items_df[["item_id", "item_category_id"]], how='left')
working.head()


# item_category_id
# item_cnt_day -> 알고싶은값

# weightontotal -> 2018 적용



,shop_id,item_id,month,item_category_id
0,5,5037,11,19
1,5,5320,11,55
2,5,5233,11,19
3,5,5232,11,23
4,5,5268,11,20


In [54]:
working["smallcateid"] = list(map(lambda x,y : str(x) +"-"+ str(y), working.month, working.item_category_id))
working.head()

,shop_id,item_id,month,item_category_id,smallcateid
0,5,5037,11,19,11-19
1,5,5320,11,55,11-55
2,5,5233,11,19,11-19
3,5,5232,11,23,11-23
4,5,5268,11,20,11-20


In [55]:
nowwork = working.merge(allinone_df[["smallcateid", "weightontotal"]], on="smallcateid", how="left")
nowwork.head()
nowwesee = nowwork.drop(["item_id", "smallcateid"], axis=1)
nowwesee.head()

,shop_id,month,item_category_id,weightontotal
0,5,11,19,0.184613
1,5,11,55,0.101616
2,5,11,19,0.184613
3,5,11,23,0.184613
4,5,11,20,0.184613


In [56]:
nowwesee.isnull().sum()
nowwesee = nowwesee.fillna(0)

In [57]:
lettest = nowwesee[["shop_id", "month", "item_category_id", "weightontotal"]].values
    
#가게별, #상품별, #월별, #카테고리별로 다를것이라는 가정 하에 VALUES를 해줌.



In [58]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [59]:
test_category_values = onehot_enc.transform(lettest[:,0:3])

In [60]:
test_real_values = minmax.transform(nowwesee["weightontotal"].values.reshape(-1,1))

In [61]:
from scipy.sparse import hstack
total_test_csc_matrix = hstack([test_category_values,csr_matrix(test_real_values)], format='csr')
total_test_csc_matrix.shape

(214200, 157)

In [62]:
y_hat_simple = lr_simple.predict(total_test_csc_matrix)

In [63]:
simple_lr_result = pd.DataFrame(y_hat_simple, columns=["item_cnt_month"])
simple_lr_result.index.name = 'ID'
simple_lr_result.to_csv("simple_lr_result.csv", sep=",")
simple_lr_result.head()

,item_cnt_month
ID,
0,1.047315
1,0.925654
2,1.047315
3,1.047315
4,1.049029


In [64]:
y_hat_lasso = lr_lasso.predict(total_test_csc_matrix)

In [67]:
lasso_lr_result = pd.DataFrame(y_hat_lasso, columns=["item_cnt_month"])
lasso_lr_result.index.name = 'ID'
lasso_lr_result.to_csv("lasso_lr_result.csv", sep=",")

print(len(lasso_lr_result.item_cnt_month.unique()))
lasso_lr_result.head()

1


,item_cnt_month
ID,
0,1.229917
1,1.229917
2,1.229917
3,1.229917
4,1.229917


In [68]:
lasso_lr_result.coef

AttributeError: 'DataFrame' object has no attribute 'coef'

In [66]:
lasso_lr_result.coef_

AttributeError: 'DataFrame' object has no attribute 'coef_'